# Data Science - Laboratorio 1
## Análisis Exploratorio, PCA y Apriori
---
**Integrantes**
- Diego Leiva
- Pablo Orellana
---

## Análisis Exploratorio
### Exploración Inicial

**Importar Librerías**

In [1]:
import os

# Manipulacion de datos
import pandas as pd
import numpy as np

# Librerias para visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

# Librerias para pre procesamiento de datos
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Librerias para clustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Libreria de metricas
from sklearn.metrics import silhouette_score, accuracy_score

**Cargar el dataset**

Fernandes,Kelwin, Cardoso,Jaime, and Fernandes,Jessica. (2017). Cervical Cancer (Risk Factors). UCI Machine Learning Repository. https://doi.org/10.24432/C5Z310.

In [2]:
data = pd.read_csv('risk_factors_cervical_cancer.csv')

**Visualizar estructura e información general del dataset**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Age                                 858 non-null    int64 
 1   Number of sexual partners           858 non-null    object
 2   First sexual intercourse            858 non-null    object
 3   Num of pregnancies                  858 non-null    object
 4   Smokes                              858 non-null    object
 5   Smokes (years)                      858 non-null    object
 6   Smokes (packs/year)                 858 non-null    object
 7   Hormonal Contraceptives             858 non-null    object
 8   Hormonal Contraceptives (years)     858 non-null    object
 9   IUD                                 858 non-null    object
 10  IUD (years)                         858 non-null    object
 11  STDs                                858 non-null    object

In [4]:
print(f"El dataset contiente: {data.shape[0]} observaciones y {data.shape[1]} variables")

El dataset contiente: 858 observaciones y 36 variables


In [5]:
data.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0


**Variables identificadas**
- Age: Edad de la paciente 
- Number.of.sexual.partners: Cantidad de parejas sexuales que ha tenido la paciente desde que inició su vida sexual 
- First.sexual.intercourse: Edad a la que tuvo el primer encuentro sexual. 
- Num.of.pregnancies: Cantidad de embarazos 
- Smokes: Si fuma o no 
- Smokes.years: Años que lleva fumando 
- Smokes.packs.per.year: Cajetillas de cigarrillos por año que fuma la paciente 
- Hormonal.Contraceptives: Si usa anticonceptivos hormonales o no 
- Hormonal.Contraceptives.years: Años que lleva usando anticonceptivos hormonales 
- IUD: Si tiene colocado algún dispositivo intrauterino (DIU) 
- IUD.years: Años que lleva usando un DIU 
- STDs: Si ha tiene enfermedades de transmisión sexual (ETS) 
- STDs.number: Cuantas ETS ha tenido 
- STDs.condylomatosis: si ha tenido condilomatosis 
- STDs.cervical.condylomatosis: si ha tenido condilomatosis cervical 
- STDs.vaginal.condylomatosis: Si ha tenido condilomatosis vaginal 
- STDs.vulvo.perineal.condylomatosis: Si ha tenido condilomatosis vulvo perineal 
- STDs.syphilis: Si ha tenido Sífilis 
- STDs.pelvic.inflammatory.disease: Si ha tenido inflamaciones pélvicas 
- STDs.genital.herpes: si ha tenido herpes genital - STDs.molluscum.contagiosum: Si ha tenido molusco contagioso 
- STDs.AIDS: Si tiene SIDA 
- STDs.HIV: Si tiene VIH 
- STDs.Hepatitis.B: si ha tenido o tiene hepatitis B 
- STDs.HPV: Si ha tenido o tiene Virus del Papiloma Humano (VPH) 
- STDs.Number.of.diagnosis: Cantidad de diagnósticos de ETS 
- STDs.Time.since.first.diagnosis: Tiempo desde el primer diagnóstico 
- STDs.Time.since.last.diagnosis: Tiempo desde el último diagnóstico 
- Dx.Cancer: Si tiene diagnóstico de cáncer o no 
- Dx.CIN: Si tiene diagnóstico de NIC (Neoplasia Intraepitelial Cervical) 
- Dx.HPV: Si tiene diagnóstico de Virus del Papiloma Humano 
- Dx: Si tiene diagnóstico 
- Pruebas para diagnosticar 
- Hinselmann: Si hicieron Colposcopía 
- Schiller: Si hicieron la prueba de Schiller 
- Citology: Si hicieron citología o no. 
- Biopsy: Si hicieron Biopsia o no 

Nota: parece que los registros que no tienen datos estan marcados con el simbolo `?` por lo tanto sera necesario modificar esos registros para tener celdas vacias o nulas.

In [6]:
data.replace('?', np.nan, inplace=True)

In [7]:
data.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
2,34,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,NaN,NaN,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0


#### Identificación de Variables

Para asegurarse de que los tipos de variables estan bien categorizados es necesaria una transformacion previa de los tipos de datos por medio de inferencia.

In [8]:
# Se convierten las variables a un tipo mas adecaudo
# por medio de inferencia
data = data.infer_objects()

# Si existen nulos la inferencia no se realizara
# por lo tanto se trata de convertir las variables
for col in data.columns:
    try:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    except ValueError:
        try:
            data[col] = pd.to_datetime(data[col], errors='coerce')
        except ValueError:
            pass

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Age                                 858 non-null    int64  
 1   Number of sexual partners           832 non-null    float64
 2   First sexual intercourse            851 non-null    float64
 3   Num of pregnancies                  802 non-null    float64
 4   Smokes                              845 non-null    float64
 5   Smokes (years)                      845 non-null    float64
 6   Smokes (packs/year)                 845 non-null    float64
 7   Hormonal Contraceptives             750 non-null    float64
 8   Hormonal Contraceptives (years)     750 non-null    float64
 9   IUD                                 741 non-null    float64
 10  IUD (years)                         741 non-null    float64
 11  STDs                                753 non-n

**Variables categóricas**

In [9]:
# Encontrar variables categoricas en el dataset de entreno
categorical = data.select_dtypes(include=['object']).columns.tolist()

print(f"Hay {len(categorical)} variables categoricas en el dataset:")
for column in categorical:
    print(f"- {column}")


# Contar las variables categoricas binarias en el dataset de entreno
binary_categorical = [col for col in categorical if data[col].nunique() == 2]

print(f"\nHay {len(binary_categorical)} variables categoricas binarias en el dataset de :")
for column in binary_categorical:
    print(f"- {column}")

Hay 0 variables categoricas en el dataset:

Hay 0 variables categoricas binarias en el dataset de :


**Frecuencia de valores categóricos**

In [10]:
# Contar las variables categoricas en el dataset de entreno
for column in categorical:
    print(data[column].value_counts())
    print('\n', '-'*35, '\n')

**Variables numéricas**

In [11]:
# Encontrar variables numericas en el dataset de entreno
numerical = data.select_dtypes(exclude=['object']).columns.tolist()

# Contar las variables cuantitativas binarias en el dataset de entreno
binary = [col for col in numerical if data[col].nunique() == 2]

# Contar las variables cuantitativas discretas en el dataset de entreno
discrete = [col for col in numerical if data[col].dtype == int and not col in binary]

# Contar las variables cuantitativas continuas en el dataset de entreno
continuous = [col for col in numerical if col not in discrete]


print(f"Hay {len(numerical)} variables cuantitativas en el dataset\n")
print(f"Hay {len(binary)} variables cuantitativas binarias en el dataset:")
for column in binary:
    print(f"- {column}")

print("\n","-"*35,"\n")

print(f"{len(discrete)} variables cuantitativas discretas:")
for column in discrete:
    print(f"- {column}")

print("\n","-"*35,"\n")

print(f"{len(continuous)} variables cuantitativas continuas:")
for column in continuous:
    print(f"- {column}")
print("-"*35)

Hay 36 variables cuantitativas en el dataset

Hay 22 variables cuantitativas binarias en el dataset:
- Smokes
- Hormonal Contraceptives
- IUD
- STDs
- STDs:condylomatosis
- STDs:vaginal condylomatosis
- STDs:vulvo-perineal condylomatosis
- STDs:syphilis
- STDs:pelvic inflammatory disease
- STDs:genital herpes
- STDs:molluscum contagiosum
- STDs:HIV
- STDs:Hepatitis B
- STDs:HPV
- Dx:Cancer
- Dx:CIN
- Dx:HPV
- Dx
- Hinselmann
- Schiller
- Citology
- Biopsy

 ----------------------------------- 

2 variables cuantitativas discretas:
- Age
- STDs: Number of diagnosis

 ----------------------------------- 

34 variables cuantitativas continuas:
- Number of sexual partners
- First sexual intercourse
- Num of pregnancies
- Smokes
- Smokes (years)
- Smokes (packs/year)
- Hormonal Contraceptives
- Hormonal Contraceptives (years)
- IUD
- IUD (years)
- STDs
- STDs (number)
- STDs:condylomatosis
- STDs:cervical condylomatosis
- STDs:vaginal condylomatosis
- STDs:vulvo-perineal condylomatosis
- ST